In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [8]:
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")

In [9]:
gdf_colerne = gdf[gdf["NAME"]=="Colerne"]
gdf = gdf.to_crs("EPSG:27700")

In [10]:
gdf['AreaPC'] = gdf['HECTARES']

In [51]:
file_info_shp = {
     "Landscape Character Areas":{
        "file_name":"G:/Wiltshire_Council/Landscape_Character_Areas/LCA_Combined_2005.shp",
        "source":"WC",
        "group":"LCA",
                                            "group_field":"Wc_lct",
                                            "groupColumnName":"Area",
                                             "groupColumnValue":False,
                       
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},
    "Ancient Tree Inventory":{
        "file_name":"G:/Woodland_Trust/Woodland_Trust_AncientTreeInventoryATI_Wilts.shp",
        "source":"WT",
        "group":"Woodland",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                             "unit":"COUNT",
                                            "unit_type":"COUNT",
                                            "count_col_name":"Id",},
    "Bedrock":{
        "file_name":"G:/British_Geological_Survey_OpenData/Bedrock Geology/625k_V5_BEDROCK_Geology_Polygons.shp",
        "source":"BGS",
       "group":"Geology",
                                            "group_field":"LEX_RCS_D",
                                            "groupColumnName":"Bedrock",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Faultlines":{
        "file_name":"G:/British_Geological_Survey_OpenData/Fault Geology/625k_V5_FAULT_Geology_Lines_Wilts.shp",
        "source":"BGS",
       "group":"Geology",
                                            "group_field":"CATEGORY",
                                            "groupColumnName":"Faultline",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH",},
    "Lakes Inventory Councils":{
        "file_name":"G:/CEH/Lakes_Inventory/CEH_Lakes_Inventory_Wilts.shp",
        "source":"CEH",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Land Cover":{
        "file_name":"G:/CEH/CEH_LCM2023/gb2023lcm.sqlite",
        "source":"CEH",
        "group":"CEH Land Cover",
                                            "group_field":"_mode",
                                            "groupColumnName":"Land Cover",
                                            "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
     
    "Flood Alert Areas":{
        "file_name":"G:/EA/Flood_Alert_Areas_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Flood Warning Areas":{
        "file_name":"G:/EA/Flood_Warning_Areas_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
      "Kennet and Avon Canals":{
        "file_name":"G:/Canals_Rivers_Trust/Canals/CRT_Kennet_Avon_Canal.shp",
        "source":"Canals Rivers Trust",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Kennet and Avon Canals Planning Buffer":{
        "file_name":"G:/Canals_Rivers_Trust/Canals/CRT_Kennet_Avon_Canal_Planning_Buffer_Wilts.shp",
        "source":"Canals Rivers Trust",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Main Rivers":{
        "file_name":"G:/EA/Statutory_Main_Rivers_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH",},  
     "Risk of Flooding":{
        "file_name":"G:/EA/Originals/Flooding/EA_RiskOfFloodingFromRiversAndSea/Risk_of_Flooding_from_Rivers_and_Sea_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                             "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "National Forest Inventory":{
        "file_name":"G:/FC_OpenData/NFI_Woodland/FC_NFI_2023_Wilts.shp",
        "source":"FC",
        "group":"Woodland",
                                            "group_field":"IFT_IOA",
                                            "groupColumnName":"Category",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},       


    "Greenspace":{
        "file_name":"G:/OS_OpenData/OS_OpenGreenspace/OS_Greenspace_Wilts.shp",
        "source":"OS",
        "group":"Greenspace",
                     "group_field":"function",
                     "groupColumnName":"Category",
                     "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},

    "Priority Habitat Inventory":{
         "file_name":"G:/NE_OpenData/PHI/NE_Priority_Habitat_Inventory_v3-11-22_Wilts.shp",
        "source":"NE",
        "group":"PHI",
                                            "group_field":"MainHabs",
                                            "groupColumnName":"Habitat",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},  
    "WSBRC Priority Habitats":{
        "file_name":"G:/WSBRC_Data/Habitats/WiltsPriorityHabs_polys_repair.shp",
        "source":"WSBRC",
        "group":"WSBRC Priority Habitats",
                                            "group_field":"PH_Name",
                                            "groupColumnName":"Priority Habitat",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},

    "Chalk Streams":{
         "file_name":"G:/EA/Chalk_Streams_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH"},

"Long-Established Woodland and Wood Pasture":{
     "file_name":"O:/NaturalEngland/Ancient Woodland Inventory Update/_Deliverable/Final/AWI_methodology_layer.shp",
     "source":"WSBRC",
     "group":"Woodland",
                                            "group_field":"Status_LEW",
                                            "groupColumnName":"Category",
                                            "groupColumnValue":{"ASNW":"Ancient Semi Natural Woodland",
                                                                "AWP": "Ancient Wood Pasture",
                                                                "IAWP":"Infilled Ancient Wood Pasture",
                                                                "LEW":"Long Established Woodland",
                                                                "PAWS":"Plantation on Ancient Woodland Site"},
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
"Landscape Biodiversity Areas":{
     "file_name":"G:/WSBRC_Data/BAP_LandscapeBiodiversityAreas/WSBRC_LandscapeBiodiversityAreas.shp",
     "source":"WSBRC",
    "group":"LBAs",
                                            "group_field":"LBA_Name",
                                            "groupColumnName":"Name",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},

   "Sustrans National Cycle Network Route":{
     "file_name":"G:/Sustrans/National_Cycle_Network/Sustrans_National_Cycle_Network_Wilts.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                                            "group_field":"Desc_",
                                            "groupColumnName":"Route type",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Sustrans_Regional Cycle Route":{
     "file_name":"G:/Sustrans/Sustrans_Regional_Cycle_Routes.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                     "group_field":"Descriptio",
                     "groupColumnName":"Route type",
                     "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Sustrans Reclassified Cycle Route":{ "file_name":"G:/Sustrans/Reclassified Routes/Sustrans_Reclassified_Routes_Wilts.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                     "group_field":"Desc_",
                     "groupColumnName":"Route type",
                     "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Public Rights of Way":{"file_name":"G:/Wiltshire_Council/Access_PROW/WiltshirePROWJan2022.shp",
     "source":"WC",
    "group":"Access and Routeways",
                      "group_field":"LYR_NAME",
                      "groupColumnName":"Access type",
                      "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Forest Roads":{
        "file_name":"G:/FC_OpenData/Management/FC_Roads_Wilts.shp",
     "source":"FC",
     "group":"Access and Routeways",
                     "group_field":False,
                     "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "CRoW S16 Dedicated Land":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Section_16_Dedicated_Land_Wilts.shp",
     "source":"NE",
        "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "CRoW_S15 Land All Types":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Section_15_Land_All_Types_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "Conclusive Registered Commons":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Conclusive_Registered_Common_Land_Wilts.shp",
     "source":"NE","group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "Conclusive Open Country":{
         "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Conclusive_Open_Country_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
     "CRoW Access Land":{
          "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Access_Land_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
                    
  "Conservation_Areas_Councils":{"file_name":"G:/Historic_England/ConservationAreas/HE_Conservation_Areas_Wilts.shp",
     "source":"HE",
      "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"sum_area_h"},  
"Country Parks":{"file_name":"G:/NE_OpenData/Country_Parks/NE_Country_Parks_Wilts.shp",
     "source":"NE",
    "group":"Access and Routeways",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
                                            
    "Crown Estates Land":{"file_name":"G:/Crown Estates/TCE_RegisteredTitle_2023Jul_Wiltshire.shp",
     "source":"NE",
        "group":"Other Land",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},  
      "WWT reserves":{
     "file_name":"G:/WSBRC_Data/BAP_LandscapeBiodiversityAreas/WSBRC_LandscapeBiodiversityAreas.shp",
     "source":"WSBRC",
     "group":"Protected Areas",
        "group_field":False,
        "groupColumnName":"Total",
        "groupColumnValue":False,
        "unit":"HECTARES",
        "unit_type":"AREA",  
        "area_col_name":"AREA"},  

    "NE_Priority_Rivers_Councils":{
          "file_name":"G:/NE_OpenData/PHI/Priority_River_Habitat_Wilts.shp",
     "source":"NE",
     "group":"Priority Habitat",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH"},           

          "NEPriorityPonds_Councils":{
              "file_name":"G:/NE_OpenData/PHI/Surveyed_Priority_Ponds_Wilts.shp",
     "source":"NE",
     "group":"Priority Habitat",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"COUNT",
                                            "unit_type":"COUNT",
                                            "count_col_name":"Site_name",},                                                                                                                                                                                                                                                                                                       
}

In [ ]:
   
   
### To do
## List
# LNR (local nature reserve)
# NNR (national nature reserve)
# RPG ()
# SAM
# SSSI
# SAC
# SPA
# WHS

## Files
#     # "Traditional Orchards":{
#     #     "file_name":"G:\NE_OpenData\PHI\NE_Priority_Habitat_Inventory_v3-11-22_Wilts.shp",
#     #     "source":"NE",
#     #     "group":"Priority Habitat",
#     #                                         "group_field":False,
#     #                                         "unit":"HECTARES",
#     #                                         "unit_type":"AREA",
#     #                                         "area_col_name":"AREA"},  



#      "NE_Woodpasture_Parkland_Councils":{"group":"Priority Habitat",
#                                             "group_field":False,
#                                             "unit":"HECTARES",
#                                             "unit_type":"AREA",
#                                             "area_col_name":"AREA"},
#     # "NFI_AT":{"group":"TBC",
#     #                                         "group_field":"IFT_IOA",
#     #                                         "unit":"HECTARES",
#     #                                         "unit_type":"AREA",
#     #                                         "area_col_name":"Area_ha"},
#     "Protected_Landscapes_Councils":{"group":"Protected Landscape",
#                                             "group_field":False,
#                                             "unit":"HECTARES",
#                                             "unit_type":"AREA",
#                                             "area_col_name":"AREA"},  
#      "RCN_Councils":{"group":"Access and Routeways",
#                                             "group_field":False,
#                                             "unit":"Metres",
#                                             "unit_type":"LENGTH",
#                                             "area_col_name":"LENGTH"},       
# #     "RIGS_Councils":{"group":"Geology",
# #                                             "group_field":False,
# #                                             "unit":"Metres",
# #                                             "unit_type":"LENGTH",
# #                                             "area_col_name":"LENGTH"},
  
# 
#     # "Hedgerows_Councils":{"group":"Priority Habitat",
    #                                         "group_field":False,
    #                                         "unit":"Metres",
    #                                         "unit_type":"LENGTH",
    #                                         "area_col_name":"LENGTH"},
    # "HE_WHS_Count_Councils":{"group":"TBC",
    #                                         "group_field":False,
    #                                         "unit":"COUNT",
    #                                         "unit_type":"COUNT",
    #                                         "area_col_name":"COUNT",},   
    #  # "HLC_Councils":{"group":"TBC",
    #  #                                        "group_field":False,
    #  #                                        "unit":"HECTARES",
    #  #                                        "unit_type":"AREA",
    #  #                                        "area_col_name":"AREA",},
    # "LCA_Councils":{"group":"Landscape Character",
    #                                         "group_field":"Wc_lct",
    #                                         "unit":"HECTARES",
    #                                         "unit_type":"AREA",
    #                                         "area_col_name":"AREA",},       
    # "MOD_Land_Councils":{"group":"Other Land",
    #                                         "group_field":False,
    #                                         "unit":"HECTARES",
    #                                         "unit_type":"AREA",
    #                                         "area_col_name":"AREA"},
    # "NCN_Councils":{"group":"Access and Routeways",
    #                                         "group_field":False,
    #                                         "unit":"Metres",
    #                                         "unit_type":"LENGTH",
    #                                         "area_col_name":"LENGTH"}, 

In [7]:
pc_name = "Colerne"

In [131]:
file_info_shp["Landscape Biodiversity Areas"]["unit"]

'Area (ha)'

In [23]:
gdf_data = gpd.read_file(file_info_shp["NEPriorityPonds_Councils"]["file_name"])

In [25]:
gdf_clipped = gdf_data.clip(gdf_local)

In [27]:
gdf_clipped.columns

Index(['Site_name', 'Survey_dat', 'x_value', 'y_value', 'Proposer', 'Project',
       'Criteria_1', 'Hcode', 'Criteria_2', 'GCN', 'Toad', 'BAP_other',
       'RDB_specie', 'NS', 'NS_sci_nam', 'NS_com_ame', 'Criteria_3',
       'Amph_Drag', 'rich_sites', 'Criteria_4', 'PSYM', 'Psym_taxa',
       'Criteria_5', 'Importance', 'comments', 'Pond', 'Type', 'Score_N',
       'Score_P', 'Notes', 'Data_Sourc', 'Pond_ID', 'NEAREST_TO', 'NUTS212NM',
       'STATIC_WAT', 'STATIC_W_1', 'STATIC_W_2', 'FID_', 'SurveyType',
       'geometry'],
      dtype='object')

In [49]:
df_all_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})


for file in file_info_shp.keys():
    gdf_data = gpd.read_file(file_info_shp[file]["file_name"])
    gdf_data = gdf_data.to_crs("EPSG:27700")
    print(file)
    df_local_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})

    for pc in gdf['NAME'].unique():
        gdf_local = gdf[gdf.NAME == pc]
        gdf_clipped = gdf_data.clip(gdf_local)
        if file_info_shp[file]["unit_type"]=="COUNT":
            gdf_dissolved = gdf_clipped[[file_info_shp[file]["count_col_name"],"geometry"]].dissolve( aggfunc="count")
            gdf_dissolved['value'] = gdf_dissolved[file_info_shp[file]["count_col_name"]]
        else:
            if file_info_shp[file]["group_field"]:
                gdf_dissolved = gdf_clipped[[file_info_shp[file]["group_field"],"geometry"]].dissolve(by = file_info_shp[file]["group_field"], aggfunc="sum").reset_index()
            else:
                gdf_dissolved = gdf_clipped[["geometry"]].dissolve( aggfunc="sum").reset_index()
            
            if file_info_shp[file]["unit_type"].lower() == "length":
                gdf_dissolved['value'] = (gdf_dissolved.geometry.length).round()
            elif file_info_shp[file]["unit_type"].lower() == "area":
                gdf_dissolved['value'] = (gdf_dissolved.geometry.area/10000).round()

        gdf_dissolved['NAME'] = pc
        if file_info_shp[file]["group_field"]:
            gdf_dissolved['groupColumnValue'] = gdf_dissolved[file_info_shp[file]["group_field"]]
        else:
            gdf_dissolved['groupColumnValue'] = np.nan

        if file_info_shp[file]["groupColumnValue"]:
            gdf_dissolved['groupColumnValue'] = gdf_dissolved['groupColumnValue'].map(file_info_shp[file]["groupColumnValue"])

        gdf_dissolved['groupColumnName'] = file_info_shp[file]["groupColumnName"]
        gdf_dissolved['mapGroup'] = file_info_shp[file]["group"]
        gdf_dissolved['unitName'] = file_info_shp[file]["unit_type"]
        gdf_dissolved['unit'] = file_info_shp[file]["unit"]
        gdf_dissolved['datasetName'] = file
        gdf_dissolved['source'] = file_info_shp[file]["source"]

        gdf_dissolved = gdf_dissolved[['NAME', "value","mapGroup","groupColumnName","groupColumnValue","unitName","unit","datasetName", "source"]]

        df_local_shp = pd.concat([df_local_shp,gdf_dissolved ])
    df_local_shp.to_csv(f"data/{file}_v1.csv")    
    df_all_shp = pd.concat([df_all_shp,df_local_shp ])
    

Crown Estates Land


In [58]:
df_all_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})


for file in file_info_shp.keys():
    df = pd.read_csv(f"data/{file}_v1.csv")
    df_all_shp = pd.concat([df_all_shp, df])

In [59]:
df_all_shp

,NAME,value,groupColumnValue,groupColumnName,mapGroup,unitName,unit,datasetName,source,Unnamed: 0
0,Wroughton,1356.0,Low Chalk Plain & Scarp,Area,LCA,AREA,Ha,Landscape Character Areas,WC,0.0
1,Wroughton,111.0,Open Downland,Area,LCA,AREA,Ha,Landscape Character Areas,WC,1.0
2,Wroughton,1285.0,Rolling Clay Lowland,Area,LCA,AREA,Ha,Landscape Character Areas,WC,2.0
3,West Swindon,249.0,Limestone Ridge,Area,LCA,AREA,Ha,Landscape Character Areas,WC,0.0
4,West Swindon,658.0,Rolling Clay Lowland,Area,LCA,AREA,Ha,Landscape Character Areas,WC,1.0
...,...,...,...,...,...,...,...,...,...,...
66,West Ashton,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0
67,Wingfield,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0
68,Great Hinton,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0
69,Keevil,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0


In [60]:
df_all_shp = pd.merge(df_all_shp, gdf[['NAME', 'AreaPC']], how = 'left', on = "NAME")

In [61]:
df_all_shp['valuePercent'] = (100*df_all_shp['value'])/df_all_shp['AreaPC']

In [62]:
df_all_ranked = pd.DataFrame({'NAME': [], 'value': [],'valuePercent': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [],'mapName': [], 'rank':[], 'rankPercent':[]})
for group in df_all_shp.groupColumnValue.unique():
    df_rank= df_all_shp[df_all_shp.groupColumnValue == group]
    df_rank_sorted = df_rank.sort_values(by = "value", ascending=False)
    df_rank_sorted['rank'] = list(range(1, df_rank_sorted.shape[0]+1))

    df_rank_percent_sorted = df_rank_sorted.sort_values(by = "valuePercent", ascending=False)
    df_rank_percent_sorted['rankPercent'] = list(range(1, df_rank_percent_sorted.shape[0]+1))
    
    df_all_ranked = pd.concat([df_all_ranked, df_rank_percent_sorted])

In [63]:
df_all_ranked['unit'] = df_all_ranked['unit'].replace("HECTARES", "Ha")
df_all_ranked['unit'] = df_all_ranked['unit'].replace("METERS", "Metres")

In [64]:
df_all_ranked.to_csv("data/outputData.csv")

In [39]:
df_all_shp

,NAME,value,groupColumnValue,groupColumnName,mapGroup,unitName,unit,datasetName,source,Unnamed: 0,AreaPC,valuePercent
0,Wroughton,2753.0,NaN,Total,Protected Areas,AREA,HECTARES,WWT reserves,WSBRC,0.0,2752.776,100.008137
1,West Swindon,910.0,NaN,Total,Protected Areas,AREA,HECTARES,WWT reserves,WSBRC,0.0,909.810,100.020883
2,Inglesham,499.0,NaN,Total,Protected Areas,AREA,HECTARES,WWT reserves,WSBRC,0.0,502.016,99.399222
3,Central Swindon South,1692.0,NaN,Total,Protected Areas,AREA,HECTARES,WWT reserves,WSBRC,0.0,1692.098,99.994208
4,Liddington,819.0,NaN,Total,Protected Areas,AREA,HECTARES,WWT reserves,WSBRC,0.0,819.309,99.962285
...,...,...,...,...,...,...,...,...,...,...,...,...
385,West Ashton,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0,792.593,0.126168
386,Wingfield,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0,817.100,0.122384
387,Great Hinton,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0,275.170,0.363412
388,Keevil,1.0,NaN,Total,Priority Habitat,COUNT,COUNT,NEPriorityPonds_Councils,NE,0.0,845.775,0.118235


In [115]:
# gdf_woodland = gpd.read_file("O:/NaturalEngland/Ancient Woodland Inventory Update/_Deliverable/Final/AWI_methodology_layer.shp")
# gdf_woodland = gpd.read_file("G:/Woodland_Trust/Woodland_Trust_AncientTreeInventoryATI_Wilts.shp")

gdf_woodland = gpd.read_file("G:/Wiltshire_Council/Access_PROW/WiltshirePROWJan2022.shp")

In [94]:
gdf_woodland

,PARISH,TYPE,REF,PATHNO,LENGTH,LYR_NAME,geometry
0,LACO,4,LACO26,26,174.0,Footpath,"LINESTRING (393113 168237, 393112 168192, 3931..."
1,EKEN,4,EKEN6,6,361.0,Footpath,"LINESTRING (411345.155 168061.955, 411210 1681..."
2,LACO,4,LACO25,25,564.0,Footpath,"LINESTRING (392833.58 168058.39, 392864.915 16..."
3,LACO,1,LACO24,24,306.0,Byway,"LINESTRING (392695 168299, 392707 168284, 3927..."
4,CORM,4,CORM19,19,435.0,Footpath,"LINESTRING (389582.625 168038.015, 389448.155 ..."
...,...,...,...,...,...,...,...
10729,None,None,None,None,0.0,None,"LINESTRING (408941.19 120556.3, 408939.7 12052..."
10730,None,None,None,None,0.0,None,"LINESTRING (410684.61 117044.52, 410722.13 117..."
10731,None,None,None,None,0.0,None,"LINESTRING (412129.19 121392.27, 412135.69 121..."
10732,None,None,None,None,0.0,None,"LINESTRING (405014.75 118742.57, 405733.53 119..."


In [16]:
gdf_woodland = gdf_nfi[gdf_nfi.CATEGORY == "Woodland"]

In [116]:
gdf_woodland_clipped = gdf_woodland.clip(gdf_colerne)

In [117]:
dis = gdf_woodland_clipped[["LYR_NAME",'geometry']].dissolve( by = "LYR_NAME",aggfunc="sum")

In [118]:
dis.geometry.length

LYR_NAME
Bridleway     6753.593029
Byway           91.387531
Footpath     23744.146277
dtype: float64

In [107]:
type_area = file_info_shp["Public Rights of Way"]["unit_type"]